<a href="https://colab.research.google.com/github/sonydata/EEG_Epilepsy_Classification/blob/main/EEG_Features_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install libraries

In [8]:
! pip install mne
! pip install antropy

import mne
import numpy as np
import pandas as pd
from scipy.signal import welch
from scipy.stats import skew, kurtosis
from antropy import sample_entropy, perm_entropy

Read EDF files

In [9]:
# Replace with the path to your EDF file
raw = mne.io.read_raw_edf('/content/sample_data/aaaaaanr_s001_t001.edf', preload=True)

# Print high-level info (channels, sampling freq, etc.)
print(raw.info)

# Access the data and sampling rate
data, times = raw.get_data(return_times=True)
sfreq = raw.info['sfreq']  # Sampling frequency
ch_names = raw.info['ch_names']
print("Sampling Frequency:", sfreq)
print("Channel Names:", ch_names)


Extracting EDF parameters from /content/sample_data/aaaaaanr_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 303499  =      0.000 ...  1213.996 secs...
<Info | 8 non-empty values
 bads: []
 ch_names: EEG FP1-LE, EEG FP2-LE, EEG F3-LE, EEG F4-LE, EEG C3-LE, EEG ...
 chs: 33 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: 2003-01-01 00:00:00 UTC
 nchan: 33
 projs: []
 sfreq: 250.0 Hz
 subject_info: <subject_info | his_id: aaaaaanr, sex: 1, last_name: aaaaaanr>
>
Sampling Frequency: 250.0
Channel Names: ['EEG FP1-LE', 'EEG FP2-LE', 'EEG F3-LE', 'EEG F4-LE', 'EEG C3-LE', 'EEG C4-LE', 'EEG A1-LE', 'EEG A2-LE', 'EEG P3-LE', 'EEG P4-LE', 'EEG O1-LE', 'EEG O2-LE', 'EEG F7-LE', 'EEG F8-LE', 'EEG T3-LE', 'EEG T4-LE', 'EEG T5-LE', 'EEG T6-LE', 'EEG FZ-LE', 'EEG CZ-LE', 'EEG PZ-LE', 'EEG OZ-LE', 'EEG PG1-LE', 'EEG PG2-LE', 'EEG EKG-LE', 'EEG SP2-LE', 'EEG SP1-LE', 'EEG 28-LE', 'EEG 29-LE', 'EEG 30-LE'

In [12]:
# Check low-level EDF header
edf_header = raw._raw_extras[0]
print("EDF Header:", edf_header)


EDF Header: {'events': [], 'units': array([1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06,
       1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06,
       1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06,
       1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06,
       1.e+00]), 'ch_names': ['EEG FP1-LE', 'EEG FP2-LE', 'EEG F3-LE', 'EEG F4-LE', 'EEG C3-LE', 'EEG C4-LE', 'EEG A1-LE', 'EEG A2-LE', 'EEG P3-LE', 'EEG P4-LE', 'EEG O1-LE', 'EEG O2-LE', 'EEG F7-LE', 'EEG F8-LE', 'EEG T3-LE', 'EEG T4-LE', 'EEG T5-LE', 'EEG T6-LE', 'EEG FZ-LE', 'EEG CZ-LE', 'EEG PZ-LE', 'EEG OZ-LE', 'EEG PG1-LE', 'EEG PG2-LE', 'EEG EKG-LE', 'EEG SP2-LE', 'EEG SP1-LE', 'EEG 28-LE', 'EEG 29-LE', 'EEG 30-LE', 'EEG T1-LE', 'EEG T2-LE', 'PHOTIC PH'], 'ch_types': ['EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG

##Extract frequency-domain features

###Power Spectral Density (PSD) and Band Power
A common approach uses the Welch method for estimating PSD. EEG signals are split into standard frequency bands. For epilepsy detection, these bands can help identify abnormal activity or changes in spectral power.

* Delta: 0.5–4 Hz

* Theta: 4–8 Hz

* Alpha: 8–12 Hz

* Beta: 12–30 Hz

* Gamma: 30–45 (or 50) Hz

**Band Power**

We can compute relative band power by dividing each band’s power by the total power.

Each band’s power (relative=True) is normalized by the total power across all frequencies, which gives more robust measures that are less sensitive to overall amplitude scaling.

In [13]:
def bandpower(data, sf, band, window_sec=4, relative=False):
    """
    Compute the average power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    sf : float
        Sampling frequency of the data.
    band : tuple
        Lower and upper frequencies of the band of interest.
    window_sec : float
        Length of each Welch segment in seconds.
    relative : bool
        If True, return the relative power (proportion of total power in the band).

    Returns
    -------
    bp : float
        Band power.
    """
    band = np.asarray(band)
    low, high = band

    # Compute Welch’s periodogram
    nperseg = int(window_sec * sf)
    freqs, psd = welch(data, sf, nperseg=nperseg)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find the indices of freqs in the band
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the power spectral density over that band
    bp = np.trapz(psd[idx_band], dx=freq_res)

    if relative:
        bp /= np.trapz(psd, dx=freq_res)
    return bp


##Time-Domain Features


Common time-domain features:

* Mean
* Variance
* Skewness
* Kurtosis
* Zero-Crossing Rate
* Teager-Kaiser Energy Operator (TKEO)

In [14]:
def compute_time_domain_features(signal):
    mean_val = np.mean(signal)
    var_val = np.var(signal)
    skew_val = skew(signal)
    kurt_val = kurtosis(signal, fisher=False)  # 'fisher=False' to match normal=3
    # Zero-crossing rate
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    zcr = len(zero_crossings) / len(signal)

    # Teager-Kaiser Energy Operator
    # TKEO(s) = x[n]^2 - x[n-1]*x[n+1]
    tkeo = np.mean(signal[1:-1]**2 - signal[:-2] * signal[2:])

    return {
        'mean': mean_val,
        'variance': var_val,
        'skewness': skew_val,
        'kurtosis': kurt_val,
        'zcr': zcr,
        'tkeo': tkeo
    }

## Entropy and Complexity Measures
Various entropy metrics are popular in seizure detection because they capture signal complexity. Two widely used measures:

* Sample Entropy (SampEn): Measures of complexity based on the regularity of a time series

* Permutation Entropy (PermEn): Looks at the order patterns of the signal values.


In [ ]:
samp_en = sample_entropy(signal, order=2, r=0.2*np.std(signal))

p_en = perm_entropy(signal, order=3, normalize=True)

## Constructing a feature vector per window

In [ ]:
def extract_features_for_channel(signal, sf):
    # Time-domain features
    td_feats = compute_time_domain_features(signal)

    # Frequency-domain features
    freq_feats = {}
    for band_name, freq_range in bands.items():
        bp = bandpower(signal, sf, freq_range, window_sec=4, relative=True)
        freq_feats[f'{band_name}_power'] = bp

    # Entropy features
    samp_en = sample_entropy(signal, order=2, r=0.2*np.std(signal))
    p_en = perm_entropy(signal, order=3, normalize=True)
    ent_feats = {
        'sample_entropy': samp_en,
        'perm_entropy': p_en
    }

    # Combine all dictionaries
    combined = {**td_feats, **freq_feats, **ent_feats}
    return combined

# Example: looping over channels and windows
all_features = []
window_size = int(4 * sfreq)  # 4-second windows, for example
overlap = 0.5  # 50% overlap

step = int(window_size * (1 - overlap))
n_samples = data.shape[1]

for ch_idx in range(data.shape[0]):
    ch_signal = data[ch_idx, :]
    start = 0
    while start + window_size < n_samples:
        windowed_signal = ch_signal[start : start + window_size]
        feats = extract_features_for_channel(windowed_signal, sfreq)
        feats['channel'] = ch_names[ch_idx]
        feats['start_sample'] = start
        feats['end_sample'] = start + window_size
        all_features.append(feats)
        start += step

df_features = pd.DataFrame(all_features)
print(df_features.head())
